In [ ]:
import os
import yaml
import itertools
import numpy as np
import pandas as pd

from query import Query

In [1]:
## System roles don't need to be defined in roles.yaml
SYSTEM_ROLES = ['ACCOUNTADMIN','SYSADMIN','SECURITYADMIN','PUBLIC','SNOWFLAKE']

## Dictionary of user/role mappings that won't be revoked, 
## regardless of what's defined in the config. 
DO_NOT_REVOKE = {
    '<USERNAME>': '<ROLE>'
}

In [2]:
## Utilities and role config enrichment

def read_yml(path):
    fd = open(path,'r')
    return yaml.load(fd)

def p(name,position):
    return name.split('.')[position]

def flatten(l):
    return list(itertools.chain.from_iterable(l))

def uppercase(o):
    if o and type(o) == list:
        return [x.upper() for x in o]
    elif o and type(o) == str:
        return o.upper()
    else:
        return o

def add_lists(lists):
    result = []
    for l in lists:
        result += l or []
    return result
    
def uppercase_role_config(role):
    role['name'] = uppercase(role['name'])
    role['parent_role'] = uppercase(role['parent_role'])
    role['read_from'] = uppercase(role['read_from'])
    role['create_in'] = uppercase(role['create_in'])
    role['owner_of'] = uppercase(role['owner_of'])
    role['warehouses'] = uppercase(role['warehouses'])
    
    return role

def is_db(name, include_partial_grants):
    if name[::-1][0] == '*' or include_partial_grants:
        return True
    else:
        return False
    
def add_all_schema(schema, schemata):
    if is_db(schema, False):
        db_name = schema[:-2]
        matches = schemata[db_name]
        result = [db_name + '.' + x for x in matches]
    else:
        result = [schema]
    return result

def get_dbs(objects, include_partial_grants):
    dbs = [p(x,0) for x in objects if is_db(x,include_partial_grants)]
    return list(set(dbs))
    

def enrich_role(role, schemata):
    creates = role['create_in'] or []
    reads   = role['read_from'] or []
    owns    = role['owner_of'] or []
    
    schema_creates = [add_all_schema(x, schemata) for x in creates] 
    schema_reads   = [add_all_schema(x, schemata) for x in reads]
    schema_owns    = [add_all_schema(x, schemata) for x in owns]
    
    role['schema_creates'] = flatten(schema_creates)
    role['schema_reads']   = flatten(schema_reads)
    role['schema_owns']    = flatten(schema_owns)
    role['db_creates']     = get_dbs(creates, False)
    role['db_reads']       = get_dbs(reads, True)   
    role['db_owns']        = get_dbs(owns, False)
    
    return role

def enrich_role_config(config, schemata):
    for role in config:
        role = uppercase_role_config(role)
        role = enrich_role(role, schemata)
    
    return config

In [3]:
## Getting existing schema and grant info

def get_schemata():
    print(">>> Getting schema...")
    result = {}
    dbs = conn.run(query="SHOW DATABASES;", return_result=True)
    
    for db in dbs:
        db_name = db['name']
        query = "SHOW SCHEMAS IN DATABASE %s" % db_name
        schemata = conn.run(query=query, return_result=True)
        result[db_name] = [s['name'] for s in schemata]
    
    print(">>> Done.")
    return result

def get_tables(schemata):
    print(">>> Getting tables...")
    all_dbs = schemata.keys()    
    all_tables = []

    for db in all_dbs:
        table_stmt = 'SHOW TABLES IN DATABASE %s;' % db
        tables = conn.run(query=table_stmt, return_result=True)
        all_tables += tables
        
        view_stmt = 'SHOW VIEWS IN DATABASE %s;' % db
        views = conn.run(query=view_stmt, return_result=True)
        all_tables += views
    
    all_tables = pd.DataFrame(all_tables)
    all_tables['db_and_schema_name'] = all_tables['database_name'] + '.' + all_tables['schema_name']
    all_tables['full_name'] = all_tables['db_and_schema_name'] + '.' + all_tables['name']
    
    print(">>> Done.")
    return all_tables

def get_grants(name, entity_type, to_or_of):
    print(">>>    Getting existing grants...")
    query = "SHOW GRANTS %s %s %s" % (to_or_of, entity_type, name)
    result = conn.run(query=query, return_result=True)
    return pd.DataFrame(result)

In [ ]:
## Check if roles' child roles have permissions that
## the parent hasn't been explicitly granted.

def get_parents(role, config):
    role_name = role['name']
    parents = [r['parent_role'] for r in config if r['name'] == role_name]
    remaining = [p for p in parents]
    
    x = 0
    while remaining != [] and x < 10:
        for r in remaining:
            grandparents = [c['parent_role'] for c in config if c['name'] == r]
            remaining.remove(r)
            parents.append(r)
            remaining += grandparents
            
            x += 1
            if x >= 10:
                print("! WARNING: %s has circular role dependencies.\n" % role_name)
                break
    
    parents = [p for p in parents if p not in SYSTEM_ROLES]
    parents = list(set(parents))
    return parents

def check_permission_inheritance(role, parents, config):
    name = role['name']
    
    db_owns        = role['db_owns']
    db_reads       = role['db_reads']
    db_creates     = role['db_creates']
    schema_owns    = role['schema_owns']
    schema_reads   = role['schema_reads']
    schema_creates = role['schema_creates']
    
    for parent in parents:
        parent_db_reads       = flatten([p['db_reads'] for p in config if p['name'] == parent])
        parent_db_creates     = flatten([p['db_creates'] for p in config if p['name'] == parent])
        parent_schema_reads   = flatten([p['schema_reads'] for p in config if p['name'] == parent])
        parent_schema_creates = flatten([p['schema_creates'] for p in config if p['name'] == parent])

        parent_db_owns         = [x for x in parent_db_reads if x in parent_db_creates]
        parent_schema_owns     = [x for x in parent_schema_reads if x in parent_schema_creates]
        
        missing_db_owns        = [x for x in db_owns if x not in parent_db_owns]
        missing_db_reads       = [x for x in db_reads if x not in parent_db_reads]
        missing_db_creates     = [x for x in db_creates if x not in parent_db_creates]
        missing_schema_owns    = [x for x in schema_owns if x not in parent_schema_owns]
        missing_schema_reads   = [x for x in schema_reads if x not in parent_schema_reads]
        missing_schema_creates = [x for x in schema_creates if x not in parent_schema_creates]
        
        for m in missing_db_owns + missing_schema_owns:
            print("! WARNING: %s explicity owns %s, but its parent %s can't create and read from it.\n" % (name, m, parent))

        for m in missing_db_reads + missing_schema_reads:
            print("! WARNING: %s can explicity read from %s, but its parent %s can't.\n" % (name, m, parent))
    
        for m in missing_db_creates + missing_schema_creates:
            print("! WARNING: %s can explicity create in %s, but its parent %s can't.\n" % (name, m, parent))

In [ ]:
## Further role validation, including circular dependencies and 
## duplicate ownership of schemata.

def get_children(config, role_name):
    children  = [r['name'] for r in config if r['parent_role'] == role_name]
    remaining = [c for c in children]
    
    x = 0
    while remaining != [] and x < 10:
        for r in remaining:
            grandchildren = [c['name'] for c in config if c['parent_role'] == r]
            remaining.remove(r)
            children.append(r)
            remaining += grandchildren
            
            x += 1
            if x >= 10:
                print("! WARNING: %s has circular role dependencies.\n" % role_name)
                break
        
    return children

def check_object(obj_type, obj_permission, role, config):
    name = role['name']
    permission = '%s_%s' % (obj_type, obj_permission)
    
    role_permissions = role[permission]
    
    children = get_children(config, name)
    
    children_permissions = [c[permission] for c in config if c['name'] in children]
    children_permissions = flatten(children_permissions)
    missing_permissions  = [x for x in children_permissions if x not in role_permissions]
    
    for m in missing_permissions:
        print("! WARNING: %s can't explicity %s from %s, but one of its children can.\n" % (name, obj_permission, m))

def check_for_duplicate_owners(config):
    owned_dbs     = [x['db_owns'] for x in all_roles]
    owned_dbs     = flatten(owned_dbs)
    owned_schemas = [x['schema_owns'] for x in all_roles]
    owned_schemas = flatten(owned_schemas)
    
    dupe_dbs      = list(set([x for x in owned_dbs if owned_dbs.count(x) > 1]))
    dupe_schemas  = list(set([x for x in owned_schemas if owned_schemas.count(x) > 1]))
    dupe_parent   = [x for x in owned_schemas if p(x,0) in owned_dbs]
    
    for d in dupe_schemas:
        print("! WARNING: Multiple owners are defined for schema %s." % d)
    
    for d in dupe_dbs:
        print("! WARNING: Multiple owners are defined for database %s." % d)
    
    for d in dupe_parent:
        print("! WARNING: Owners are defined for both schema %s and database %s." % (d, p(d,0)))

def validate_role_config(config):
    print(">>> Checking role dependencies...")
    check_for_duplicate_owners(config)
    
    for role in config:
        parents = get_parents(role,all_roles)
        check_permission_inheritance(role, parents, all_roles)

In [ ]:
## Check current permissions against current grants.

def validate_permission(row, role_config):
    grantee_name = row['grantee_name']
    object_type = row['granted_on']
    name = row['name']
    privilege = row['privilege']

    sc = role_config['schema_creates']
    sr = role_config['schema_reads']
    so = role_config['schema_owns']
    dc = role_config['db_creates']
    dr = role_config['db_reads']
    do = role_config['db_owns']

    ## Warehouses
    if object_type == 'WAREHOUSE' and privilege == 'USAGE':
        check = name in role_config['warehouses']
    
    ## Databases
    elif object_type == 'DATABASE' and privilege == 'USAGE':
        check = name in (dc + dr + do)
    
    elif object_type == 'DATABASE' and privilege == 'CREATE SCHEMA':
        check = name in (dc + do)
    
    elif object_type == 'DATABASE' and privilege == 'OWNERSHIP':
        check = name in do
    
    ## Schema
    elif object_type == 'SCHEMA' and privilege == 'OWNERSHIP':
        check = name in (so + [s for s in sc if s in sr])
        
    elif object_type == 'SCHEMA' and privilege == 'USAGE':
        check = name in (sc + sr + so)

    elif object_type == 'SCHEMA' and privilege == 'CREATE TABLE':
        check = name in (sc + so)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE VIEW':
        check = name in (sc + so)

    elif object_type == 'SCHEMA' and privilege == 'MODIFY':
        check = False

    elif object_type == 'SCHEMA' and privilege == 'MONITOR':
        check = False
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE EXTERNAL TABLE':
        check = False
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE FILE FORMAT':
        check = False
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE FUNCTION':
        check = False
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE PIPE':
        check = False
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE PROCEDURE':
        check = False
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE SEQUENCE':
        check = False

    elif object_type == 'SCHEMA' and privilege == 'CREATE STAGE':
        check = False
    
    ## Tables
    elif object_type == 'TABLE' and privilege == 'OWNERSHIP':
        schema = p(name,0) + '.' + p(name,1)
        check = schema in (sc + so)
            
    elif object_type == 'VIEW' and privilege == 'OWNERSHIP':
        schema = p(name,0) + '.' + p(name,1)
        check = schema in (sc + so)

    elif object_type == 'TABLE' and privilege == 'SELECT':
        schema = p(name,0) + '.' + p(name,1)
        check = schema in sr

    elif object_type == 'VIEW' and privilege == 'SELECT':
        schema = p(name,0) + '.' + p(name,1)
        check = schema in sr
    
    ## Other
    elif object_type == 'ROLE':
        return True
    
    else:
        print("! WARNING: Unknown permission found: %s of %s %s" % (privilege, object_type, name))
        check = 'MISSING'
    return check

In [ ]:
## Check for existing roles and users that
## aren't defined in the config.

def find_undefined(objects, object_type):
    stmt = "SHOW %ss;\n" % object_type
    
    current_result = conn.run(query=stmt, return_result=True)
    current_df = pd.DataFrame(current_result)
    current = current_df['name']
    target = [o['name'] for o in objects]
    missing = np.setdiff1d(current,target)
    missing = [m for m in missing if m not in SYSTEM_ROLES]
    
    for m in missing:
        print("! WARNING: %s %s is missing from %ss.yaml." % (object_type, m, object_type))

In [ ]:
def check_role(role):
    role_name = role['name']
    print(">>>    Checking current grants...")
    df = get_grants(role_name,'role','to')
    
    if len(df > 0):
        df['matches_target'] = df.apply(validate_permission, args=(role,), axis=1)
        df['full_schema_name'] = df.apply(add_schema_name, axis=1) 
    
    return df

In [ ]:
def add_schema_name(row):
    object_type = row['granted_on']
    name = row['name']
    
    if object_type in ('TABLE','SCHEMA','VIEW'):
        schema_name = p(name,0) + '.' + p(name,1)
    else:
        schema_name = None
    
    return schema_name

In [ ]:
def initialize_role(role):
    name = role['name']
    desc = role['description']
    
    stmt =  "CREATE ROLE IF NOT EXISTS %s;\n" % name
    stmt += "ALTER ROLE IF EXISTS %s SET COMMENT = '%s';\n" % (name, desc)
    
    conn.run(query=stmt, return_result=False)

def set_parent(role):
    name = role['name']
    parent = role['parent_role']
    
    stmt = ""
    
    df = get_grants(name, 'ROLE', 'OF')
    for index, row in df.iterrows():
        if row['granted_to'] == 'ROLE': ## Don't revoke user grants
            stmt += "REVOKE ROLE %s FROM ROLE %s;" % (row['role'], row['grantee_name'])
    
    stmt += "GRANT ROLE %s TO ROLE %s;" % (name, parent)
    
    conn.run(query=stmt, return_result=False)

In [ ]:
def revoke_mismatches(df):
    statements = []
    to_revoke = df[df['matches_target'] == False]
    
    for index, row in to_revoke.iterrows():
        revoke_statement = write_revoke(row)
        statements.append(revoke_statement)
    
    statements = list(set(statements))
    
    return ''.join(statements)

In [ ]:
def write_revoke(row):
    
    grantee_name = row['grantee_name']
    object_type = row['granted_on']
    name = row['name']
    privilege = row['privilege']
    
    ## Warehouses
    if object_type == 'WAREHOUSE' and privilege == 'USAGE':
        stmt = "REVOKE USAGE ON WAREHOUSE %s FROM %s;\n" % (name, grantee_name)
    
    ## Databases
    elif object_type == 'DATABASE' and privilege == 'USAGE':
        stmt = "REVOKE USAGE ON DATABASE %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'DATABASE' and privilege == 'CREATE SCHEMA':
        stmt = "REVOKE CREATE SCHEMA ON DATABASE %s FROM %s;\n" % (name, grantee_name)
    
    ## Schemas
    elif object_type == 'SCHEMA' and privilege == 'OWNERSHIP':
        stmt = "REVOKE OWNERSHIP ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
        
    elif object_type == 'SCHEMA' and privilege == 'CREATE':
        stmt = "REVOKE CREATE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
        
    elif object_type == 'SCHEMA' and privilege == 'CREATE TABLE':
        stmt = "REVOKE CREATE TABLE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE VIEW':
        stmt = "REVOKE CREATE VIEW ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'USAGE':
        stmt = "REVOKE USAGE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)

    elif object_type == 'SCHEMA' and privilege == 'MODIFY':
        stmt = "REVOKE MODIFY ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
        
    elif object_type == 'SCHEMA' and privilege == 'MONITOR':
        stmt = "REVOKE MONITOR ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE EXTERNAL TABLE':
        stmt = "REVOKE CREATE EXTERNAL TABLE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE FILE FORMAT':
        stmt = "REVOKE CREATE FILE FORMAT ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE FUNCTION':
        stmt = "REVOKE CREATE FUNCTION ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE PIPE':
        stmt = "REVOKE CREATE PIPE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE PROCEDURE':
        stmt = "REVOKE CREATE PROCEDURE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE SEQUENCE':
        stmt = "REVOKE CREATE SEQUENCE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)

    elif object_type == 'SCHEMA' and privilege == 'CREATE STAGE':
        stmt = "REVOKE CREATE STAGE ON SCHEMA %s FROM %s;\n" % (name, grantee_name)
    
    ## Tables
    elif object_type == 'TABLE' and privilege == 'SELECT':
        schema = p(name,0) + '.' + p(name,1)
        stmt = "REVOKE SELECT ON ALL TABLES IN SCHEMA %s FROM %s;\n" % (schema, grantee_name)

    elif object_type == 'VIEW' and privilege == 'SELECT':
        schema = p(name,0) + '.' + p(name,1)
        stmt = "REVOKE SELECT ON ALL VIEWS IN SCHEMA %s FROM %s;\n" % (schema, grantee_name)
    
    ## Other
    else:
        print("! WARNING: Unknown revoke requested on %s for %s %s\n" % (privilege, object_type, name))
        stmt = "-- WTF LOL\n;"
    
    return stmt

In [ ]:
def make_target_grants(role_config):
    js = []
    
    name = role_config['name']
    desc = role_config['description']
    parent = role_config['parent_role']
    
    warehouses     = role_config['warehouses']
    schema_creates = role_config['schema_creates']
    schema_reads   = role_config['schema_reads']
    schema_owns    = role_config['schema_owns']
    db_creates     = role_config['db_creates']
    db_reads       = role_config['db_reads']
    db_owns        = role_config['db_owns']
    
    schema_creates = [s for s in schema_creates if 'INFORMATION_SCHEMA' not in s]
    schema_reads   = [s for s in schema_reads if 'INFORMATION_SCHEMA' not in s]
    
    for wh in warehouses:
        js.append( {'name': wh, 'granted_on': 'WAREHOUSE', 'privilege': 'USAGE'} )
    
    for sc in schema_creates:
        js.append( {'name': p(sc,0), 'granted_on': 'DATABASE', 'privilege': 'USAGE'} )
        js.append( {'name': sc, 'granted_on': 'SCHEMA', 'privilege': 'USAGE'} )
        js.append( {'name': sc, 'granted_on': 'SCHEMA', 'privilege': 'CREATE TABLE'} )
        js.append( {'name': sc, 'granted_on': 'SCHEMA', 'privilege': 'CREATE VIEW'} )

    for sr in schema_reads:
        js.append( {'name': p(sr,0), 'granted_on': 'DATABASE', 'privilege': 'USAGE'} )
        js.append( {'name': sr, 'granted_on': 'SCHEMA', 'privilege': 'USAGE'} )
        js.append( {'name': sr, 'granted_on': 'SCHEMA', 'privilege': 'SELECT'} )

    for so in schema_owns:
        js.append( {'name': p(so,0), 'granted_on': 'DATABASE', 'privilege': 'USAGE'} )
        js.append( {'name': so, 'granted_on': 'SCHEMA', 'privilege': 'OWNERSHIP'} )
        
    for dc in db_creates:
        js.append( {'name': dc, 'granted_on': 'DATABASE', 'privilege': 'USAGE'} )
        js.append( {'name': dc, 'granted_on': 'DATABASE', 'privilege': 'CREATE SCHEMA'} )
    
    for dr in db_reads:
        js.append( {'name': dr, 'granted_on': 'DATABASE', 'privilege': 'USAGE'} )

    for do in db_owns:
        js.append( {'name': do, 'granted_on': 'DATABASE', 'privilege': 'USAGE'} )
        js.append( {'name': do, 'granted_on': 'DATABASE', 'privilege': 'CREATE SCHEMA'} )
        js.append( {'name': do, 'granted_on': 'DATABASE', 'privilege': 'OWNERSHIP'} )
    
    for row in js:
        row['grantee_name'] = name
    
    return pd.DataFrame(js)

def check_permission_for_all_tables(full_schema_name, current_grants, privilege, all_tables):
    current_objects = current_grants.loc[(current_grants['privilege'] == privilege) & (current_grants['full_schema_name'] == full_schema_name)]
    current_objects = list(current_objects['name'])    

    existing_objects = all_tables.loc[all_tables['db_and_schema_name'] == full_schema_name]
    existing_objects = list(existing_objects['full_name'])    

    missing_objects = [x for x in existing_objects if x not in current_objects]
        
    if len(missing_objects) > 0:
        exists = False
    else:
        exists = True
    
    return exists

def check_existing_permissions(row, current_grants, all_tables):
    grantee_name = row['grantee_name']
    object_type = row['granted_on']
    name = row['name']
    privilege = row['privilege']

    if privilege == 'OWNERSHIP' and object_type == 'SCHEMA':
        exists = check_permission_for_all_tables(name, current_grants, 'OWNERSHIP', all_tables)
        
    elif privilege == 'SELECT' and object_type == 'SCHEMA':
        exists = check_permission_for_all_tables(name, current_grants, 'SELECT', all_tables)
    
    else:
        df = current_grants.loc[
            (current_grants['grantee_name'] == grantee_name) &
            (current_grants['granted_on'] == object_type) &
            (current_grants['name'] == name) &
            (current_grants['privilege'] == privilege)
        ]
        
        if len(df) == 0:
            exists = False
        else:
            exists = True
            
    return exists

def removing_existing_permissions(current_grants, target_grants, all_tables):
    target_grants['exists'] = target_grants.apply(check_existing_permissions, args=(current_grants, all_tables, ), axis=1)
    return target_grants[target_grants['exists'] != True]

def write_grants(df):
    statements = []
    
    for index, row in df.iterrows():
        grant_statement = write_grant_statemnt(row)
        statements.append(grant_statement)
    
    statements = list(set(statements))
    
    return ''.join(statements)
    
def write_grant_statemnt(row):
    grantee_name = row['grantee_name']
    object_type = row['granted_on']
    name = row['name']
    privilege = row['privilege']
    
    ## Warehouses
    if object_type == 'WAREHOUSE' and privilege == 'USAGE':
        stmt = "GRANT USAGE ON WAREHOUSE %s TO %s;\n" % (name, grantee_name)
    
    ## Databases
    elif object_type == 'DATABASE' and privilege == 'CREATE SCHEMA':
        stmt = "GRANT CREATE SCHEMA ON DATABASE %s TO ROLE %s;\n" % (name, grantee_name)

    elif object_type == 'DATABASE' and privilege == 'USAGE':
        stmt = "GRANT USAGE ON DATABASE %s TO ROLE %s;\n" % (name, grantee_name)

    elif object_type == 'DATABASE' and privilege == 'OWNERSHIP':
        stmt = "GRANT OWNERSHIP ON DATABASE %s TO ROLE %s COPY CURRENT GRANTS;\n" % (name, grantee_name)

    ## Schemas
    elif object_type == 'SCHEMA' and privilege == 'OWNERSHIP':
        stt1 = "GRANT OWNERSHIP ON SCHEMA %s TO ROLE %s COPY CURRENT GRANTS;\n" % (name, grantee_name)
        stt2 = "GRANT OWNERSHIP ON ALL TABLES IN SCHEMA %s TO ROLE %s COPY CURRENT GRANTS;\n" % (name, grantee_name)
        stt3 = "GRANT OWNERSHIP ON ALL VIEWS IN SCHEMA %s TO ROLE %s COPY CURRENT GRANTS;\n" % (name, grantee_name)
        stmt = stt1 + stt2 + stt3
        
    elif object_type == 'SCHEMA' and privilege == 'CREATE TABLE':
        stmt = "GRANT CREATE TABLE ON SCHEMA %s TO ROLE %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'CREATE VIEW':
        stmt = "GRANT CREATE VIEW ON SCHEMA %s TO ROLE %s;\n" % (name, grantee_name)
    
    elif object_type == 'SCHEMA' and privilege == 'USAGE':
        stmt = "GRANT USAGE ON SCHEMA %s TO ROLE %s;\n" % (name, grantee_name)

    elif object_type == 'SCHEMA' and privilege == 'SELECT':
        stt1 = "GRANT SELECT ON ALL TABLES IN SCHEMA %s TO ROLE %s;\n" % (name, grantee_name)
        stt2 = "GRANT SELECT ON ALL VIEWS IN SCHEMA %s TO ROLE %s;\n" % (name, grantee_name)
        stmt = stt1 + stt2
    
    ## Other
    else:
        print("! WARNING: Unknown grant requested on %s for %s %s\n" % (privilege, object_type, name))
        stmt = "-- WTF LOL"
    
    return stmt

In [ ]:
def enrich_user_config(all_users):
    for user in all_users:
        user = enrich_user(user)
    
    return all_users
        
def enrich_user(user):
    user['name'] = uppercase(user['name'])
    user['roles'] = uppercase(user['roles'])
    
    props = user['properties']
    
    props['default_warehouse'] = uppercase(props['default_warehouse'])
    props['default_namespace'] = uppercase(props['default_namespace'])
    props['default_role'] = uppercase(props['default_role'])
    
    return user

def validate_user_config(user_config, role_config):
    all_roles = [r['name'] for r in role_config]
    all_roles += SYSTEM_ROLES
    
    for user in user_config:
        validate_user(all_roles, user)
    
def validate_user(all_roles, user):
    name = user['name']
    user_roles = user['roles'] 
    default = user['properties']['default_role']

    if default not in user_roles:
        print("! WARNING: %s default role not in assigned roles" % name)
    
    if default not in all_roles:
            print("! WARNING: %s default role %s not in roles.yaml" % (name, default))
        
    for ur in user_roles:
        if ur not in all_roles:
            print("! WARNING: %s default role %s not in roles.yaml" % (name, ur))

def initialize_user(user):
    name = user['name']
    properties = user['properties']
    unquoted = ['disable_mfa']
    
    props =  ''.join(["%s='%s' " %x for x in properties.items() if x[0] not in unquoted])
    props += ''.join(["%s=%s " %x for x in properties.items() if x[0] in unquoted])
    
    stmt = "ALTER USER IF EXISTS %s SET %s;\n" % (name,  props)
    
    conn.run(query=stmt, return_result=False)

def assign_user_roles(user):
    name = user['name']
    target_roles = user['roles']
    
    df = get_grants(name,'user','to')
    
    if len(df > 0):
        current_roles = df['role']
        revoke_current_roles(name, current_roles)
    
    grant_user_roles(name, target_roles)

def revoke_current_roles(name, roles):
    for r in roles:
        if DO_NOT_REVOKE.get(name) != r:
            stmt = "REVOKE ROLE %s FROM USER %s;\n" % (r, name)
            conn.run(query=stmt, return_result=False)
        
def grant_user_roles(name, roles):
    for r in roles:
        stmt = "GRANT ROLE %s TO USER %s;\n" % (r, name)
        conn.run(query=stmt, return_result=False)

In [ ]:
def configure_role(role, all_tables):
    print(">>> Setting up role %s" % role['name'])
    
    ## Create role and description
    initialize_role(role)
    set_parent(role)
    
    ## Check current and new grants
    current_grants = check_role(role)
    target_grants = make_target_grants(role)

    if len(current_grants > 0):
        new_grants = removing_existing_permissions(current_grants, target_grants, all_tables)
    else:
        new_grants = target_grants
    
    ## If new grants, add them
    if len(new_grants > 0):
        grants = write_grants(new_grants)
        print(">>>    Writing %i grants..." % grants.count(';'))
        print(grants)
        conn.run(query=grants,return_result=False)
        
        ## Update current grants again
        current_grants = check_role(role)
    else:
        print(">>>    --- No grants to add.")
    
    ## Get revokes from current grants
    revokes = revoke_mismatches(current_grants)
    if len(revokes) > 0:
        print(">>>    Revoking %i grants..." % revokes.count(';'))
        print(revokes)
        conn.run(query=revokes, return_result=False)
    else:
        print(">>>    --- No grants to revoke.")
    
    print(">>> %s complete." % role['name'])

def configure_user(user):
    print(">>> Setting up user %s..." % user['name'])
    initialize_user(user)
    
    print(">>>    Reassigning role grants...")
    assign_user_roles(user)
    
    print(">>> %s complete." % user['name'])

In [ ]:
## BEGIN SCRIPT

conn = Query()

In [ ]:
schemata = get_schemata()
tables = get_tables(schemata)

In [ ]:
all_roles = read_yml('roles.yaml')
all_users = read_yml('users.yaml')

all_roles = enrich_role_config(all_roles, schemata)
all_users = enrich_user_config(all_users)

validate_role_config(all_roles)
validate_user_config(all_users, all_roles)

find_undefined(all_users, 'user')
find_undefined(all_roles, 'role')

# Define roles first
for role in all_roles:
    configure_role(role, tables)

for user in all_users:
    configure_user(user)